# Abstract Base Classes (ABCs), Protocols and Mixins

## 1. Introduction

Abstract Base Classes (ABCs) and Protocols are both ways to define and enforce interfaces in Python. They help ensure that a class implements particular methods, promoting a form of "design by contract". However, they serve different purposes and are suited to different scenarios.

## 2. Understanding Abstract Base Classes

An Abstract Base Class in Python is a class that cannot be instantiated on its own and usually contains one or more abstract methods. An abstract method is a method that is declared, but contains no implementation. ABCs are part of the `abc` module. To create an ABC, you need to import `ABC` and `abstractmethod` from the `abc` module. 

In [4]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    @abstractmethod
    def start_engine(self):
        pass

    @abstractmethod
    def stop_engine(self):
        pass

Abstract methods are declared in the abstract base class but do not contain any implementation. Subclasses must provide an implementation for these methods.

In [5]:
class Car(Vehicle):
    def start_engine(self):
        print("Car engine started")
    
    def stop_engine(self):
        print("Car engine stopped")
        
class Motorcycle(Vehicle):
    pass

car = Car()
motorcycle = Motorcycle()
# This raises an error because the Motorcycle class does not implement the start_engine method
motorcycle.start_engine()

TypeError: Can't instantiate abstract class Motorcycle with abstract methods start_engine, stop_engine

**Plugin Example**

Consider a scenario where you are developing a plugin system. You want to ensure that all plugins follow a specific interface.

In [ ]:
from abc import ABC, abstractmethod

class Plugin(ABC):
    @abstractmethod
    def load(self):
        pass
    
    @abstractmethod
    def execute(self):
        pass
    
    @abstractmethod
    def unload(self):
        pass

class AudioPlugin(Plugin):
    def load(self):
        print("Loading audio plugin")
    
    def execute(self):
        print("Executing audio plugin")
    
    def unload(self):
        print("Unloading audio plugin")

class VideoPlugin(Plugin):
    def load(self):
        print("Loading video plugin")
    
    def execute(self):
        print("Executing video plugin")
    
    def unload(self):
        print("Unloading video plugin")

# Usage
plugins = [AudioPlugin(), VideoPlugin()]
for plugin in plugins:
    plugin.load()
    plugin.execute()
    plugin.unload()

Loading audio plugin
Executing audio plugin
Unloading audio plugin
Loading video plugin
Executing video plugin
Unloading video plugin


**ABCs: Class Decorators**

Class decorators can also be used to register classes as implementing an ABC.

In [ ]:
from abc import ABC, ABCMeta

class PluginBase(ABC):
    pass

class PluginRegistry(ABCMeta):
    def __init__(cls, name, bases, namespace):
        super().__init__(name, bases, namespace)
        if not hasattr(cls, 'registry'):
            cls.registry = []
        cls.registry.append(cls)

class Plugin(PluginBase, metaclass=PluginRegistry):
    pass

class MyPlugin(Plugin):
    pass

print(Plugin.registry)

[<class '__main__.Plugin'>, <class '__main__.MyPlugin'>]


## 3. Introduction to Protocols

Protocols, introduced in PEP 544 and available from Python 3.8 onwards, allow you to define interfaces using structural subtyping (also known as duck typing). This means a class can conform to a protocol if it has the required methods and attributes, without explicitly inheriting from it.

Protocols are defined using the `Protocol` class from the `typing` module.

In [ ]:
from typing import Protocol

class Drivable(Protocol):
    def start_engine(self) -> None:
        pass
    
    def stop_engine(self) -> None:
        pass

Unlike ABCs, a class can conform to a protocol simply by implementing the required methods, without needing to inherit from the protocol.

In [ ]:
class Car(Drivable):
    def start_engine(self) -> None:
        print("Car engine started")
    
    def stop_engine(self) -> None:
        print("Car engine stopped")

In [ ]:
class Boat:
    def start_engine(self):
        print("Boat engine started")
    
    def stop_engine(self):
        print("Boat engine stopped")

In [ ]:
def start_and_stop(vehicle: Drivable):
    vehicle.start_engine()
    vehicle.stop_engine()

# These will work even though Boat and Car do not inherit from Drivable
boat = Boat()
start_and_stop(boat)

car = Car()
start_and_stop(car)

Boat engine started
Boat engine stopped
Car engine started
Car engine stopped


Protocols are particularly useful for type checking in large codebases. They allow you to specify the required interface without imposing inheritance constraints.

In [ ]:
from typing import Protocol, List

class Savable(Protocol):
    def save(self) -> None:
        pass

class DatabaseRecord:
    def save(self):
        print("Saving record to the database")

class FileRecord:
    def save(self):
        print("Saving record to the file")

def batch_save(records: List[Savable]):
    for record in records:
        record.save()

# This will work with both DatabaseRecord and FileRecord
records = [DatabaseRecord(), FileRecord()]
batch_save(records)

Saving record to the database
Saving record to the file


**Protocols: Runtime Checkable Protocols**

Protocols can be made runtime checkable using the `@runtime_checkable` decorator from the `typing` module.

In [ ]:
from typing import Protocol, runtime_checkable

@runtime_checkable
class Drivable(Protocol):
    def start_engine(self) -> None:
        pass
    
    def stop_engine(self) -> None:
        pass

class Truck:
    def start_engine(self):
        print("Truck engine started")
    
    def stop_engine(self):
        print("Truck engine stopped")

truck = Truck()
print(isinstance(truck, Drivable))

True


## 4. Mixins

A Mixin is a class that provides methods to other classes through inheritance, but it is not intended to stand alone. Mixins allow you to add specific, reusable functionalities to classes in a way that promotes code reuse and separation of concerns. 

**Key Characteristics of Mixins**

1. **Purpose-specific**: Mixins should focus on providing a specific piece of functionality.
2. **Not intended for standalone use**: A Mixin should not be instantiated on its own.
3. **Can be combined with other classes**: Mixins are typically used with other base classes to extend their functionality.

Let's consider a practical example to illustrate how Mixins work. Suppose you have a system where different types of objects need to be serialized to JSON and XML formats.

In [ ]:
import json
import xml.etree.ElementTree as ET

class JsonSerializerMixin:
    def to_json(self):
        return json.dumps(self.__dict__)

class XmlSerializerMixin:
    def to_xml(self):
        root = ET.Element(self.__class__.__name__)
        for key, value in self.__dict__.items():
            child = ET.SubElement(root, key)
            child.text = str(value)
        return ET.tostring(root, encoding='unicode')

class Person(JsonSerializerMixin, XmlSerializerMixin):
    def __init__(self, name, age):
        self.name = name
        self.age = age

# Usage
person = Person(name="John Doe", age=30)
print(person.to_json())
print(person.to_xml()) 

{"name": "John Doe", "age": 30}
<Person><name>John Doe</name><age>30</age></Person>


In this example:
- `JsonSerializerMixin` provides a `to_json` method.
- `XmlSerializerMixin` provides a `to_xml` method.
- `Person` class inherits from both mixins, gaining the ability to serialize to both JSON and XML.

Mixins can be combined with ABCs to enforce a certain interface while providing concrete methods. Here’s how you can do that:

In [ ]:
from abc import ABC, abstractmethod
import json

class Serializer(ABC):
    @abstractmethod
    def serialize(self):
        pass

class JsonSerializerMixin:
    def serialize(self):
        return json.dumps(self.__dict__)

class XmlSerializerMixin:
    def serialize(self):
        root = ET.Element(self.__class__.__name__)
        for key, value in self.__dict__.items():
            child = ET.SubElement(root, key)
            child.text = str(value)
        return ET.tostring(root, encoding='unicode')

class Person(JsonSerializerMixin, Serializer):
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Animal(XmlSerializerMixin, Serializer):
    def __init__(self, species, age):
        self.species = species
        self.age = age

# Usage
person = Person(name="John Doe", age=30)
print(person.serialize()) 

animal = Animal(species="Cat", age=5)
print(animal.serialize()) 

{"name": "John Doe", "age": 30}
<Animal><species>Cat</species><age>5</age></Animal>


In this example:
- `Serializer` is an ABC with an abstract `serialize` method.
- `JsonSerializerMixin` and `XmlSerializerMixin` provide concrete implementations of `serialize`.
- `Person` and `Animal` classes inherit from both a mixin and the `Serializer` ABC, ensuring they have a `serialize` method.


Mixins are a powerful tool in Python's inheritance model that promote code reuse and separation of concerns. They complement Abstract Base Classes and Protocols by providing reusable methods and behaviors, making your classes more flexible and modular.

## 5. Comparing Abstract Base Classes and Protocols

**When to Use ABCs**:
- **Strict Interface Enforcement**: Use ABCs when you need strict control over the interface and want to ensure that subclasses explicitly declare their intent to conform to the interface.
- **Complex Inheritance Structures**: Use ABCs when dealing with complex inheritance structures where you need multiple inheritance or mixins.

**When to Use Protocols**:
- **Duck Typing**: Use Protocols when you prefer structural subtyping (duck typing) and want classes to conform to an interface without explicit inheritance.
- **Type Checking**: Use Protocols to take advantage of type hints and static type checking in tools like `mypy`.

**When to Use Mixins**:
- **Mixins**: Use Mixins to add reusable functionality to classes. Mixins can be used alongside ABCs and Protocols to provide concrete methods or additional behavior.